In [3]:
import os

In [1]:
def generate_data_generator_for_two_images(genX1):
	while True:
		X1i = genX1.next()
		yield X1i[0], X1i[1]

In [2]:
# Parameters
epochs = 20
batch_size = 20
impl = 2 			# gpu	

###########################################
# Data

img_width, img_height, channels = 224, 224, 3 		# Resolution of inputs
input_shape = 4096

dataset = 'SASE-FE'
partition = 'prealigned'


In [4]:
if dataset == 'OULU-CASIA':
	train_data_dir = os.path.join('..', '..', '_Dataset', dataset, 'consecutive', 'training')	
	validation_data_dir = os.path.join('..', '..', '_Dataset', dataset, 'consecutive', 'validation')
	frames = 5
	n_output = 6
	nb_train_samples = 5941 / batch_size
	nb_validation_samples = 2025 / batch_size
	
else:
	partition = ''

	train_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'training')	
	validation_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'validation')

	frames = 5
	n_output = 12

	nb_train_samples = 27971 / batch_size
	nb_validation_samples = 4173 / batch_size


In [6]:
dataset = 'SASE-FE'
partition = 'prealigned'

train_data_dir = os.path.join('..','..', '_Dataset', dataset, '5frames', 'training')
validation_data_dir = os.path.join('..','..', '_Dataset', dataset, '5frames', 'validation')

frames = 5
n_output = 12

nb_train_samples = 27971 / batch_size
nb_validation_samples = 4173 / batch_size


In [7]:
train_data_dir

'../../_Dataset/SASE-FE/5frames/training'

In [8]:
data_dir = '/Users/dim__gag/Desktop/SASE-FE/FakeTrue_DB'

train_data_dir = os.path.join(data_dir, '5frames', 'training')
validation_data_dir = os.path.join(data_dir, '5frames', 'validation')

In [9]:
train_data_dir

'/Users/dim__gag/Desktop/SASE-FE/FakeTrue_DB/5frames/training'

In [12]:
# split the data into training and validation from the directory with videos SASE-FE
train_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'training')
validation_data_dir = os.path.join('..', '..', '_Dataset', dataset, '5frames', 'validation')

frames = 5

# number of classes


In [13]:
train_data_dir

'../../_Dataset/SASE-FE/5frames/training'

In [25]:
data_dir = '/Users/dim__gag/Desktop/SASE-FE/FakeTrue_DB'

participants = os.listdir(data_dir)



In [26]:
# data.remove('.DS_Store')

In [27]:
participants

['Pavel',
 'Alena',
 'Merilin',
 'yiiri',
 'Dona',
 'Victor',
 'Suman',
 'Hassan',
 'Airiin',
 'Polina',
 '.DS_Store',
 'Toyib',
 'Alexander',
 'Elmar',
 'Karl Gregori',
 'Roxanne',
 'Sameer',
 'Kirill',
 'Lukas',
 'Remo',
 'Anton',
 'Asif',
 'Nikita',
 'Vladimir',
 'Rezwan',
 'Iiris',
 'Vladimiz',
 'anee',
 'Pejman',
 'Reena',
 'age',
 'Umesh',
 'Laur',
 'Karl',
 'Laura',
 'sinle',
 'Francisca',
 'Mari-liis',
 'Nina',
 'Darwin',
 'Helen',
 'Richard',
 'Andreas',
 'Ivan',
 'Yeh',
 'Aleksander Makarov',
 'Laura Jogede',
 'Lembit',
 'Ahmed',
 'Mate',
 'Teddy']

In [28]:
from collections import defaultdict
import os
import numpy as np
import shutil

In [29]:
# print the directories of the data from SASE-Fe dataset
data_dir = '/Users/dim__gag/Desktop/SASE-FE/FakeTrue_DB'
participants = os.listdir(data_dir)
print(participants)


['Pavel', 'Alena', 'Merilin', 'yiiri', 'Dona', 'Victor', 'Suman', 'Hassan', 'Airiin', 'Polina', '.DS_Store', 'Toyib', 'Alexander', 'Elmar', 'Karl Gregori', 'Roxanne', 'Sameer', 'Kirill', 'Lukas', 'Remo', 'Anton', 'Asif', 'Nikita', 'Vladimir', 'Rezwan', 'Iiris', 'Vladimiz', 'anee', 'Pejman', 'Reena', 'age', 'Umesh', 'Laur', 'Karl', 'Laura', 'sinle', 'Francisca', 'Mari-liis', 'Nina', 'Darwin', 'Helen', 'Richard', 'Andreas', 'Ivan', 'Yeh', 'Aleksander Makarov', 'Laura Jogede', 'Lembit', 'Ahmed', 'Mate', 'Teddy']


In [32]:
# Print the classes of every directory
for x in participants:
    classes = os.listdir(data_dir + '/' + x)
    print(classes)
    

['D2N2Sur.MP4', 'N2S.MP4', 'N2D.MP4', 'N2C.MP4', 'N2A.MP4', 'S2N2H.MP4', 'N2H.MP4', 'H2N2A.MP4', 'N2Sur.MP4', 'H2N2C.MP4', 'H2N2D.MP4', 'H2N2S.MP4']
['D2N2Sur.MP4', 'N2S.MP4', 'N2D.MP4', 'N2C.MP4', 'N2A.MP4', 'S2N2H.MP4', 'N2H.MP4', 'H2N2A.MP4', 'N2Sur.MP4', 'H2N2C.MP4', 'H2N2D.MP4', 'H2N2S.MP4']
['D2N2Sur.MP4', 'N2S.MP4', 'N2D.MP4', 'N2C.MP4', 'N2A.MP4', 'S2N2H.MP4', 'N2H.MP4', 'H2N2A.MP4', 'N2Sur.MP4', 'H2N2C.MP4', 'H2N2D.MP4', 'H2N2S.MP4']
['D2N2Sur.MP4', 'N2S.MP4', 'N2D.MP4', 'N2C.MP4', 'N2A.MP4', 'S2N2H.MP4', 'N2H.MP4', 'H2N2A.MP4', 'N2Sur.MP4', 'H2N2C.MP4', 'H2N2D.MP4', 'H2N2S.MP4']
['D2N2Sur.MP4', 'N2S.MP4', 'N2D.MP4', 'N2C.MP4', 'N2A.MP4', 'S2N2H.MP4', 'N2H.MP4', 'H2N2A.MP4', 'N2Sur.MP4', 'H2N2C.MP4', 'H2N2D.MP4', 'H2N2S.MP4']
['D2N2Sur.MP4', 'N2S.MP4', 'N2D.MP4', 'N2C.MP4', 'N2A.MP4', 'S2N2H.MP4', 'N2H.MP4', 'H2N2A.MP4', 'N2Sur.MP4', 'H2N2C.MP4', 'H2N2D.MP4', 'H2N2S.MP4']
['D2N2Sur.MP4', 'N2S.MP4', 'N2d.MP4', 'N2c.MP4', 'N2A.MP4', 'S2N2H.MP4', 'N2H.MP4', 'H2N2a.MP4', 'N2Sur.MP

In [31]:
# Drop .DS_Store from directory
participants.remove('.DS_Store')

In [33]:
# Set the names of the classes
classes = ['Fake', 'True']

# Create the directories for the training and validation data
if not os.path.exists(os.path.join(data_dir, '5frames', 'training')):
    os.makedirs(os.path.join(data_dir, '5frames', 'training'))
if not os.path.exists(os.path.join(data_dir, '5frames', 'validation')):
    os.makedirs(os.path.join(data_dir, '5frames', 'validation'))

# Create the directories for the classes in the training and validation data
for x in classes:
    if not os.path.exists(os.path.join(data_dir, '5frames', 'training', x)):
        os.makedirs(os.path.join(data_dir, '5frames', 'training', x))
    if not os.path.exists(os.path.join(data_dir, '5frames', 'validation', x)):
        os.makedirs(os.path.join(data_dir, '5frames', 'validation', x))
        

In [ ]:
# Is this the right way to split the data into training and validation?
'''# Split the data into training and validation
for x in participants:
    for y in classes:
        # Get the list of all images with emotion
        images = os.listdir(data_dir + '/' + x + '/' + y)
        # Split the images into training and validation
        training = [i for i in images if i.startswith('train')]
        validation = [i for i in images if i.startswith('val')]
        # Copy the images into the training and validation directories
        for i in training:
            source = os.path.join(data_dir, x, y, i)
            destination = os.path.join(data_dir, '5frames', 'training', y, i)
            shutil.copyfile(source, destination)
        for i in validation:
            source = os.path.join(data_dir, x, y, i)
            destination = os.path.join(data_dir, '5frames', 'validation', y, i)
            shutil.copyfile(source, destination)

# Print the number of training and validation images
for x in classes:
    print('Total images in training ' + x + ' data:', len(os.listdir(os.path.join(data_dir, '5frames', 'training', x))))
    print('Total images in validation ' + x + ' data:', len(os.listdir(os.path.join(data_dir, '5frames', 'validation', x))))
'''